In [1]:
'''
MIT License

Copyright (c) 2019 hsjplus

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

from keras.layers import Input, Dense, Flatten
from keras.datasets.mnist import load_data
from keras.layers.convolutional import Conv2D, MaxPooling2D

import tensorflow as tf
from random import shuffle
import numpy as np

Using TensorFlow backend.


In [2]:
# load data
data = load_data(path='mnist.npz')

X_train = data[0][0] # (60000, 28, 28)
X_train = np.reshape(X_train, [-1, 28, 28, 1])
y_train = data[0][1] # (60000, )
X_test = data[1][0] # (10000, 28, 28)
X_test = np.reshape(X_test, [-1, 28, 28, 1])
y_test = data[1][1] # (10000, )

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
# model - keras codes
x = Input(shape=(28, 28, 1))
conv2d_1 = Conv2D(filters=32, kernel_size=[3, 3])(x)
conv2d_2 = Conv2D(filters=32, kernel_size=[3, 3])(conv2d_1)
pool = MaxPooling2D(pool_size=[2,2], strides=[2, 2])(conv2d_2)

flatten = Flatten()(pool)
output = Dense(10)(flatten)

W0620 01:04:00.056898 140290782128000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0620 01:04:00.095274 140290782128000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 01:04:00.102996 140290782128000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0620 01:04:00.143348 140290782128000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [4]:
# output : tensor type
print(type(x))
print(type(output))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [0]:
# model - tensorflow codes 
y = tf.placeholder(dtype=tf.int32, shape=[None])
# output is from keras output
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# evaluation
prediction = tf.argmax(output, axis=-1, output_type=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(x=prediction, y=y), dtype=tf.float32))

In [0]:
epoch = 5
batch_size = 256
idx_list = [i for i in range(len(X_train))]

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
for i in range(epoch):
  cost_list = []
  acc_list = []
  
  for step in range(int(len(X_train) / batch_size)):
    shuffle(idx_list)
    # x is from keras output
    _, cost, acc = sess.run([optimizer, loss, accuracy], feed_dict={x: X_train[idx_list[:batch_size]], 
                                                     y: y_train[idx_list[:batch_size]]})
    cost_list.append(cost)
    acc_list.append(acc)
    
  print('epoch {} - loss: {:.4f}, acc: {:.4f}'.format(i, np.average(cost_list), np.average(acc_list)))

epoch 0 - loss: 1.6022, acc: 0.8891
epoch 1 - loss: 0.2399, acc: 0.9556
epoch 2 - loss: 0.1561, acc: 0.9681
epoch 3 - loss: 0.1306, acc: 0.9728
epoch 4 - loss: 0.1162, acc: 0.9747


In [8]:
sess.run(accuracy, feed_dict={x: X_test, y: y_test})

0.9637